Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [17]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


train_path=os.path.join('/tmp/h-or-s')

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epochs,logs={}):
    if logs.get('acc')>0.999:
      print("Reached 99.9% accuracy so cancelling training!")
      self.model.stop_training=True
 

callbacks = myCallback()

--2019-07-04 11:38:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-07-04 11:38:07 (169 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 147, 147, 16)      2320      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 71, 71, 32)        4640      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 33, 33, 64)       

In [30]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(300,300),
    batch_size=16,
    class_mode='binary'
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [31]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    epochs=15,
    verbose=1,
    steps_per_epoch=5,
    callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
5/5 [==============================] - 6s 1s/step - loss: 1.6324 - acc: 0.5125
Epoch 2/15
5/5 [==============================] - 6s 1s/step - loss: 0.7242 - acc: 0.7375
Epoch 3/15
5/5 [==============================] - 6s 1s/step - loss: 0.4481 - acc: 0.8000
Epoch 4/15
5/5 [==============================] - 6s 1s/step - loss: 0.2330 - acc: 0.9000
Epoch 5/15
5/5 [==============================] - 6s 1s/step - loss: 0.4359 - acc: 0.8875
Epoch 6/15
5/5 [==============================] - 6s 1s/step - loss: 0.1734 - acc: 0.9625
Epoch 7/15
5/5 [==============================] - 6s 1s/step - loss: 0.0661 - acc: 0.9875
Epoch 8/15
5/5 [==============================] - 6s 1s/step - loss: 0.0264 - acc: 1.0000
